In [0]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
# To run 1.X code, unmodified (except for contrib), in TensorFlow 2.0
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

In [0]:
data_text = """The unanimous Declaration of the thirteen united States of America, When in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature and of Nature's God entitle them, a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation.
We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness.--That to secure these rights, Governments are instituted among Men, deriving their just powers from the consent of the governed, --That whenever any Form of Government becomes destructive of these ends, it is the Right of the People to alter or to abolish it, and to institute new Government, laying its foundation on such principles and organizing its powers in such form, as to them shall seem most likely to effect their Safety and Happiness. Prudence, indeed, will dictate that Governments long established should not be changed for light and transient causes; and accordingly all experience hath shewn, that mankind are more disposed to suffer, while evils are sufferable, than to right themselves by abolishing the forms to which they are accustomed. But when a long train of abuses and usurpations, pursuing invariably the same Object evinces a design to reduce them under absolute Despotism, it is their right, it is their duty, to throw off such Government, and to provide new Guards for their future security.--Such has been the patient sufferance of these Colonies; and such is now the necessity which constrains them to alter their former Systems of Government. The history of the present King of Great Britain is a history of repeated injuries and usurpations, all having in direct object the establishment of an absolute Tyranny over these States. To prove this, let Facts be submitted to a candid world.
He has refused his Assent to Laws, the most wholesome and necessary for the public good.
He has forbidden his Governors to pass Laws of immediate and pressing importance, unless suspended in their operation till his Assent should be obtained; and when so suspended, he has utterly neglected to attend to them.
He has refused to pass other Laws for the accommodation of large districts of people, unless those people would relinquish the right of Representation in the Legislature, a right inestimable to them and formidable to tyrants only.
He has called together legislative bodies at places unusual, uncomfortable, and distant from the depository of their public Records, for the sole purpose of fatiguing them into compliance with his measures.
He has dissolved Representative Houses repeatedly, for opposing with manly firmness his invasions on the rights of the people.
He has refused for a long time, after such dissolutions, to cause others to be elected; whereby the Legislative powers, incapable of Annihilation, have returned to the People at large for their exercise; the State remaining in the mean time exposed to all the dangers of invasion from without, and convulsions within.
He has endeavoured to prevent the population of these States; for that purpose obstructing the Laws for Naturalization of Foreigners; refusing to pass others to encourage their migrations hither, and raising the conditions of new Appropriations of Lands.
He has obstructed the Administration of Justice, by refusing his Assent to Laws for establishing Judiciary powers.
He has made Judges dependent on his Will alone, for the tenure of their offices, and the amount and payment of their salaries.
He has erected a multitude of New Offices, and sent hither swarms of Officers to harrass our people, and eat out their substance.
He has kept among us, in times of peace, Standing Armies without the Consent of our legislatures.
He has affected to render the Military independent of and superior to the Civil power.
He has combined with others to subject us to a jurisdiction foreign to our constitution, and unacknowledged by our laws; giving his Assent to their Acts of pretended Legislation:
For Quartering large bodies of armed troops among us:
For protecting them, by a mock Trial, from punishment for any Murders which they should commit on the Inhabitants of these States:
For cutting off our Trade with all parts of the world:
For imposing Taxes on us without our Consent:
For depriving us in many cases, of the benefits of Trial by Jury:
For transporting us beyond Seas to be tried for pretended offences
For abolishing the free System of English Laws in a neighbouring Province, establishing therein an Arbitrary government, and enlarging its Boundaries so as to render it at once an example and fit instrument for introducing the same absolute rule into these Colonies:
For taking away our Charters, abolishing our most valuable Laws, and altering fundamentally the Forms of our Governments:
For suspending our own Legislatures, and declaring themselves invested with power to legislate for us in all cases whatsoever.
He has abdicated Government here, by declaring us out of his Protection and waging War against us.
He has plundered our seas, ravaged our Coasts, burnt our towns, and destroyed the lives of our people.
He is at this time transporting large Armies of foreign Mercenaries to compleat the works of death, desolation and tyranny, already begun with circumstances of Cruelty & perfidy scarcely paralleled in the most barbarous ages, and totally unworthy the Head of a civilized nation.
He has constrained our fellow Citizens taken Captive on the high Seas to bear Arms against their Country, to become the executioners of their friends and Brethren, or to fall themselves by their Hands.
He has excited domestic insurrections amongst us, and has endeavoured to bring on the inhabitants of our frontiers, the merciless Indian Savages, whose known rule of warfare, is an undistinguished destruction of all ages, sexes and conditions.
In every stage of these Oppressions We have Petitioned for Redress in the most humble terms: Our repeated Petitions have been answered only by repeated injury. A Prince whose character is thus marked by every act which may define a Tyrant, is unfit to be the ruler of a free people.
Nor have We been wanting in attentions to our Brittish brethren. We have warned them from time to time of attempts by their legislature to extend an unwarrantable jurisdiction over us. We have reminded them of the circumstances of our emigration and settlement here. We have appealed to their native justice and magnanimity, and we have conjured them by the ties of our common kindred to disavow these usurpations, which, would inevitably interrupt our connections and correspondence. They too have been deaf to the voice of justice and of consanguinity. We must, therefore, acquiesce in the necessity, which denounces our Separation, and hold them, as we hold the rest of mankind, Enemies in War, in Peace Friends.
We, therefore, the Representatives of the united States of America, in General Congress, Assembled, appealing to the Supreme Judge of the world for the rectitude of our intentions, do, in the Name, and by Authority of the good People of these Colonies, solemnly publish and declare, That these United Colonies are, and of Right ought to be Free and Independent States; that they are Absolved from all Allegiance to the British Crown, and that all political connection between them and the State of Great Britain, is and ought to be totally dissolved; and that as Free and Independent States, they have full Power to levy War, conclude Peace, contract Alliances, establish Commerce, and to do all other Acts and Things which Independent States may of right do. And for the support of this Declaration, with a firm reliance on the protection of divine Providence, we mutually pledge to each other our Lives, our Fortunes and our sacred Honor."""


In [11]:
# Text cleaning

# 1. Converting entire string to lower case.
# 2. Removing the '(apostrophe) symbol.
# 3. Remove Punctuations from string.
# 4. Remove short words from string.

import re

def text_cleaner(text):
    # lower case text
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    # remove punctuations
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    long_words=[]
    # remove short word
    for i in newString.split():
        if len(i)>=3:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()

# preprocess the text
data_new = text_cleaner(data_text)

data_new

'the unanimous declaration the thirteen united states america when the course human events becomes necessary for one people dissolve the political bands which have connected them with another and assume among the powers the earth the separate and equal station which the laws nature and nature god entitle them decent respect the opinions mankind requires that they should declare the causes which impel them the separation hold these truths self evident that all men are created equal that they are endowed their creator with certain unalienable rights that among these are life liberty and the pursuit happiness that secure these rights governments are instituted among men deriving their just powers from the consent the governed that whenever any form government becomes destructive these ends the right the people alter abolish and institute new government laying its foundation such principles and organizing its powers such form them shall seem most likely effect their safety and happiness pr

In [18]:
# Creating Sequences

# The way this problem is modeled is we take in 30 characters as context and ask the model to predict the next character. 
# We essentially need enough characters in the input sequence that your model is able to get the context.

def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
          # Example, let's say length of text is 100 characters, then
          # for i in range(30, 100) we will make 70 sequences
          # for i=30 --> sequence will be [30-30: 30+1] --> [0:31]
          # for i=31 --> sequence will be [31-30: 31+1] --> [1:32]
          # for i=32 --> sequence will be [32-30: 32+1] --> [2:33]
          
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

# create sequences   
sequences = create_seq(data_new)

Total Sequences: 7052


In [19]:
sequences[:5]

['the unanimous declaration the t',
 'he unanimous declaration the th',
 'e unanimous declaration the thi',
 ' unanimous declaration the thir',
 'unanimous declaration the thirt']

In [20]:
# Encoding Sequences
# Where each character in corpus (i.e. dataset) will be given a number 
# This encoding will be in alphabetical order, i.e. if the corpus contains 
# 5 alphabets namely [c,g,a,u,i], then the encoding will be
# a-1, c-2, g-3, i-4, u-5

# create a character mapping index
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

print (mapping)

# Encoding the pre generated sequence.
def encode_seq(seq):
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences

# encode the sequences
sequences = encode_seq(sequences)

# Visualizing the first sequence
print (sequences[0])


{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
[20, 8, 5, 0, 21, 14, 1, 14, 9, 13, 15, 21, 19, 0, 4, 5, 3, 12, 1, 18, 1, 20, 9, 15, 14, 0, 20, 8, 5, 0, 20]


In [21]:
# Creating training and validation data
from sklearn.model_selection import train_test_split

# vocabulary size (available characters in the corpus)
# This can be (<=26) if the language is english, and can grow further for other languages.
vocab = len(mapping)

# Converting sequence list to array.
sequences = np.array(sequences)

# create X and y i.e. input and output data
# input  --> all characters in sequence expect last one, i.e. 29 characters
# output ---> the last character of the sequence.
X, y = sequences[:,:-1], sequences[:,-1]

# one hot encode y, convert y to categorical value (classification problem not regression)
y = to_categorical(y, num_classes=vocab)

# create train and validation sets, test size is 10%
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (6346, 30) Val shape: (706, 30)


In [22]:
# Building the Neuaral N/W model

# define model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# fit the model
model.fit(X_tr, y_tr, epochs=100, verbose=2, validation_data=(X_val, y_val))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            1350      
_________________________________________________________________
gru_1 (GRU)                  (None, 150)               90450     
_________________________________________________________________
dense_1 (Dense)              (None, 27)                4077      
Total params: 95,877
Trainable params: 95,877
Non-trainable params: 0
_________________________________________________________________
None


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 6346 samples, validate on 706 samples
Epoch 1/100





 - 10s - loss: 2.7440 - acc: 0.1992 - val_loss: 2.3878 - val_acc: 0.3017
Epoch 2/100
 - 9s - 

In [0]:
# Inference from Model

# Generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):

# model: the sequential model we trained earlier
# mapping: to encode the input characters to integers using the mapping function
# seq_length: the length of input sequence
# seed_text: input text given to model, after which it need to predict the characters
# n_chars:  the length of output we want to generate

    in_text = seed_text
    # generate a fixed number of characters

    for _ in range(n_chars):
        # encode the characters as integers
        print ("input sequence at iteration {}: ".format(_), in_text)

        encoded = [mapping[char] for char in in_text]
        
        # truncate sequences to a fixed length
        # we want the input of length 30, there can be 3 possibilities:
          # 1. Given sequence is more than 30 in length - pad_sequence will truncate the initial part and keep last 30 characters
          # 2. Given sequence is equal to 30 in length - pad_sequence will not do anything
          # 3. Given sequence is less than 30 in length - pad_sequence will add 0's to the initial part to make length 30 characters

        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
          if index == yhat:
            out_char = char
            break
        # append to input
        in_text += char
    return in_text

In [45]:
inp = "large armies of"
print ("output Sequence:", generate_seq(model, mapping, 30, inp.lower(), 10))

input sequence at iteration 0:  large armies of
input sequence at iteration 1:  large armies off
input sequence at iteration 2:  large armies offi
input sequence at iteration 3:  large armies offic
input sequence at iteration 4:  large armies office
input sequence at iteration 5:  large armies offices
input sequence at iteration 6:  large armies offices 
input sequence at iteration 7:  large armies offices a
input sequence at iteration 8:  large armies offices an
input sequence at iteration 9:  large armies offices and
output Sequence: large armies offices and 
